# **Taller 3**
### **Integrantes:**
* Diego Felipe Carvajal Lombo (201911910)
* Brenda Catalina Barahona Pinilla (201812721)
* Sergio Julian Zona Moreno (201914936)


# **Carga y muestreo de los datos**

In [2]:
# Libraries
seed = 2023
import pandas as pd
import numpy as np
import sqlite3
import numpy as np

In [3]:
# Dependencias
# !apt-get install -y build-essential
# !apt-get install manpages-dev
# !apt-get install -y gcc
# !pip install -U scikit-learn
# !pip install scikit-surprise requests tmdbsimple

## **Carga de datos**

In [4]:
# Read a portion of the CSV file into a pandas DataFrame
def load_csv(path):    
    #chunk_size = 50000
    df = pd.read_csv(path, encoding='utf-8')
    return df

In [5]:
df_links = load_csv('content/links.csv')
df_links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [6]:
df_movies = load_csv('content/movies.csv')
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
df_ratings = load_csv('content/ratings.csv')
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [8]:
df_tags = load_csv('content/tags.csv')
df_tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [9]:
# Columnas de cada DF
print("Links:", df_links.columns.tolist())
print("----------------------------------------")
print("Movies:", df_movies.columns.tolist())
print("----------------------------------------")
print("Ratings:", df_ratings.columns.tolist())
print("----------------------------------------")
print("Tags:", df_tags.columns.tolist())

Links: ['movieId', 'imdbId', 'tmdbId']
----------------------------------------
Movies: ['movieId', 'title', 'genres']
----------------------------------------
Ratings: ['userId', 'movieId', 'rating', 'timestamp']
----------------------------------------
Tags: ['userId', 'movieId', 'tag', 'timestamp']


# **Perfilamiento y entendimiento de los datos**
Obtendremos estadísticas descriptivas pertinentes y posteriormente ingresaremos el conjunto de datos a Pandas Profiling para obtener un reporte adecuado de correlación e interacción entre variables.

In [10]:
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [11]:
df_l = df_links.dropna()
df_l.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9734 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9734 non-null   int64  
 1   imdbId   9734 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 304.2 KB


La Tabla df_l no tiene nulos en la columna tmbdId, además sus ids son valores entetos, lo que permite la fácil interacción con el API de esta DB

In [12]:
df_l = df_l.astype(np.int64)
df_l.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9734 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   movieId  9734 non-null   int64
 1   imdbId   9734 non-null   int64
 2   tmdbId   9734 non-null   int64
dtypes: int64(3)
memory usage: 304.2 KB


Debido a que Neo4j no permite tener nodos con valores diferentes, se va a eliminar de los ratings y movies las películas con el mismo nombre y año de publicación.

Un ejemplo de estas son las películas "Emma (1996)", que tienen exactamente el mismo nombre pero diferente id

In [13]:
ejemplo_duplicado = df_movies[df_movies['title'] == 'Emma (1996)']
ejemplo_duplicado

,movieId,title,genres
650,838,Emma (1996),Comedy|Drama|Romance
5601,26958,Emma (1996),Romance


In [14]:
df_movies.shape

(9742, 3)

In [15]:
ejemplo_no_duplicado = df_movies[df_movies['title'] == 'Emma (1996)']
ejemplo_no_duplicado

,movieId,title,genres
650,838,Emma (1996),Comedy|Drama|Romance
5601,26958,Emma (1996),Romance


In [16]:
df_movies = df_movies.drop_duplicates(subset = ['title'], keep = 'last')
df_movies.shape

(9737, 3)

Con esto vimos que se eliminaron 5 películas

Ahora, vamos a eliminar las películas del DF df_movies que no tienen un ID correspondiente en TMDB. Esto debido a que vamos a trabajar con esta API.

In [17]:
df_movies = df_movies[df_movies['movieId'].isin(df_l['movieId'])]
df_movies.shape

(9731, 3)

In [18]:
df_movies.to_csv('df_m.csv', encoding='utf-8')

Con esto, obtuvimos un Dataframe que no tiene películas repetidas por nombre y año de publicación, y además, tiene un Id en TMDB

Ahora vamos a procedes a elimiar las reviews, links y tags de las películas que no se encuentren en el DF df_movies. Con esto aseguramos que los datos sean consistentes entre los dataframes

## Consistencia en DF: Links

In [19]:
df_l.shape

(9734, 3)

In [20]:
df_l = df_l[df_l['movieId'].isin(df_movies['movieId'])]
df_l.shape

(9731, 3)

## Consistencia en DF: Ratings

In [21]:
num_unique_movies = df_ratings['movieId'].nunique()

# Imprimir el resultado
print("Número de movieId diferentes:", num_unique_movies)

Número de movieId diferentes: 9724


In [22]:
df_ratings.shape

(100836, 4)

In [23]:
df_ratings = df_ratings[df_ratings['movieId'].isin(df_movies['movieId'])]
df_ratings.shape

(100728, 4)

# Creación de los modelos


En esta sección se diseñarán los modelos que serán implementados en el front de la aplicación. Algunos de ellos son On-Premise, pero otros son entrenados previamente e importados.

## Modelo de filtrado colaborativo

In [61]:
from surprise import Dataset, Reader, KNNWithZScore, accuracy
from surprise.model_selection import train_test_split

df = df_ratings[['userId', 'movieId', 'rating']]

#Cargar datos
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df, reader)

# Split data
trainset, testset = train_test_split(data, test_size=0.2, random_state=seed)

In [98]:
from surprise.model_selection import GridSearchCV

param_grid = {'k': [10, 20, 30, 40, 50],
              'sim_options': {'name': ['pearson','pearson_baseline'], # pearson, pearson_baseline
                              'user_based': [False],
                              'verbose': [False],  
                             },
              'verbose': [False]
             }
# Set up the grid search
gs = GridSearchCV(KNNWithZScore, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)

# Fit the grid search on the dataset
gs.fit(trainset)

# Print the best score and parameters
print('Best score:', gs.best_score['rmse'])
print('Best parameters:', gs.best_params['rmse'])

Best score: 0.8852908000023623
Best parameters: {'k': 40, 'sim_options': {'name': 'pearson_baseline', 'user_based': False, 'verbose': False}, 'verbose': False}


In [162]:
# Save the model
from surprise import dump

# Factorization model
#knn = KNNWithZScore(k = 10, sim_options = {'name': 'pearson', 'user_based': False, 'verbose': False}, 'verbose': False})
knn = KNNWithZScore(k = gs.best_params['rmse']['k'], sim_options=gs.best_params['rmse']['sim_options'], random_state=seed, verbose=False)
knn.fit(trainset)

dump.dump('knn.pkl', algo=knn)

# Enriquecimiento Semántico

Se utilizará la página web themoviedb.org para proveer un APIKey y con esto obtener mayor información de las películas para realizar el enriquecimiento semántico


In [24]:
import requests

url = "https://api.themoviedb.org/3/movie/550?api_key=e41c8a68517883ed8a2d66aca50b3371"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"adult":false,"backdrop_path":"/hZkgoQYus5vegHoetLkCJzb17zJ.jpg","belongs_to_collection":null,"budget":63000000,"genres":[{"id":18,"name":"Drama"},{"id":53,"name":"Thriller"},{"id":35,"name":"Comedy"}],"homepage":"http://www.foxmovies.com/movies/fight-club","id":550,"imdb_id":"tt0137523","original_language":"en","original_title":"Fight Club","overview":"A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground \"fight clubs\" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.","popularity":70.038,"poster_path":"/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg","production_companies":[{"id":508,"logo_path":"/7cxRWzi4LsVm4Utfpr1hfARNurT.png","name":"Regency Enterprises","origin_country":"US"},{"id":711,"logo_path":"/tEiIH5QesdheJmDAqQwvtN60727.png","name":"Fox 2000 Pictures","origin_country":"US"},{"id":20555,"logo_path":"/hD8yEGUBl

Utilizaremos el wrapper TMDB simple para obtener más información de las películas

https://github.com/celiao/tmdbsimple

In [25]:
import tmdbsimple as tmdb
tmdb.API_KEY = 'e41c8a68517883ed8a2d66aca50b3371'

tmdb.REQUESTS_TIMEOUT = 5  # seconds, for both connect and read

In [26]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [27]:
df_m_extended = df_movies.copy()
df_m_extended = df_m_extended.drop(['genres'], axis=1)
df_m_extended

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017)
9738,193583,No Game No Life: Zero (2017)
9739,193585,Flint (2017)
9740,193587,Bungo Stray Dogs: Dead Apple (2018)


In [28]:
from tqdm import tqdm

In [ ]:
for row in tqdm(df_l.itertuples(name="Link")):
    movieId,_,tmdbId = row.movieId, row.imdbId, int(row.tmdbId)
    movie = tmdb.Movies(tmdbId)
    actors = np.nan
    director = np.nan
    keyWords = np.nan
    sim_mov = np.nan
    watch_prov = np.nan
    
    try:
        actors = [c["name"] for c in movie.credits()["cast"]]
        director = [c["name"] for c in filter(lambda x: x["job"]=="Director", movie.crew)][0]

        #SERENDIPIA

        keyWords = [kw["name"] for kw in movie.keywords()["keywords"]]
        sim_mov = [s["title"] for s in movie.similar_movies()["results"]]
        watch_prov = [wp["provider_name"] for wp in movie.watch_providers()["results"]["US"]["flatrate"]]

    except Exception as e:
        print(f"the movie with id: {movieId}, and tmdbId: {tmdbId} was not found.")
        print(e)

    #GUARDAR EN DATA SET EXTENDIDO
    df_m_extended.loc[df_m_extended['movieId'] == movieId, 'actors'] = ', '.join(actors[:5]) if actors is not np.nan else np.nan
    df_m_extended.loc[df_m_extended['movieId'] == movieId, 'director'] = director if director is not np.nan else np.nan
    df_m_extended.loc[df_m_extended['movieId'] == movieId, 'keyWords'] = ', '.join(keyWords[:5]) if keyWords is not np.nan else np.nan
    df_m_extended.loc[df_m_extended['movieId'] == movieId, 'similarMovies'] = ', '.join(sim_mov[:5]) if sim_mov is not np.nan else np.nan
    df_m_extended.loc[df_m_extended['movieId'] == movieId, 'watchProviders'] = ', '.join(watch_prov) if watch_prov is not np.nan else np.nan

#json.dump(tmdbmovies,currentdatapath/"tmdbmovies.json")
#tmdbmovies[:5]

In [ ]:
df_m_extended

In [153]:
df_m_extended.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'movieId', 'title', 'actors', 'director',
       'keyWords', 'similarMovies', 'watchProviders'],
      dtype='object')

In [158]:
df_m_extended = df_m_extended.loc[df_m_extended['actors'].isna() == False]

In [159]:
df_m_extended.to_csv('df_movies.csv', encoding='utf-8', index=False)

# Ontological data

In [31]:
df_m_extended = pd.read_csv('df_movies.csv')
df_m_extended

,movieId,title,actors,director,keyWords,similarMovies,watchProviders
0,1,Toy Story (1995),"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,"martial arts, jealousy, friendship, bullying, ...","Police Story, Harlem Nights, Snow White: A Tal...",Disney Plus
1,2,Jumanji (1995),"Robin Williams, Kirsten Dunst, Bradley Pierce,...",Joe Johnston,"giant insect, board game, jungle, disappearanc...","Medicine Man, The Jungle Book, Honey, I Shrunk...","fuboTV, Peacock Premium, AMC Plus Apple TV Cha..."
2,3,Grumpier Old Men (1995),"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,"fishing, halloween, sequel, old man, best friend","Young Guns II, Death to Smoochy, The Nutty Pro...",NaN
3,4,Waiting to Exhale (1995),"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"based on novel or book, interracial relationsh...","Just Cause, Love Story, The Sword in the Stone...",NaN
4,5,Father of the Bride Part II (1995),"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,"parent child relationship, baby, midlife crisi...","10, Little Man, Young Guns II, S.W.A.T., Welco...","Disney Plus, Hulu"
...,...,...,...,...,...,...,...
9571,193581,Black Butler: Book of the Atlantic (2017),"Daisuke Ono, Maaya Sakamoto, Yukari Tamura, Ju...",Noriyuki Abe,"butler, victorian england, based on manga, dar...","Seven Days War, The Time Machine, To Walk Invi...",NaN
9572,193583,No Game No Life: Zero (2017),"Yoko Hikasa, Ai Kayano, Yoshitsugu Matsuoka, K...",Atsuko Ishizuka,"based on novel or book, anti hero, magic, post...","10, Just Cause, Love Story, The Sword in the S...",HiDive
9573,193585,Flint (2017),"Queen Latifah, Betsy Brandt, Rob Morrow, Marin...",Bruce Beresford,NaN,NaN,NaN
9574,193587,Bungo Stray Dogs: Dead Apple (2018),"Kensho Ono, Mamoru Miyano, Kisho Taniyama, Yut...",Takuya Igarashi,"supernatural, mafia, based on manga, super pow...","Donnie Brasco, Thir13en Ghosts, Ghosts of War,...",Funimation Now


In [32]:
import pandas as pd
import re

def preprocess_movie_name(movie_name):
    # Remove parentheses and their contents using regular expressions
    processed_name = re.sub(r'\([^()]*\)', '', movie_name)
    
    # Remove leading/trailing whitespace
    processed_name = processed_name.strip()
    processed_name = processed_name.replace(' ','_')
    
    return processed_name

def process_titles(df):
    processed_titles = []
    for index, row in df.iterrows():
        movie_name = row['title']
        processed_name = preprocess_movie_name(movie_name)
        processed_titles.append(processed_name)

    df['processed_titles'] = processed_titles
    df = df[['movieId', 'title', 'processed_titles', 'actors', 'director', 'keyWords', 'similarMovies', 'watchProviders']]
    return df

df_o_movies = df_m_extended.copy()
df_o_movies = process_titles(df_o_movies)
df_o_movies

,movieId,title,processed_titles,actors,director,keyWords,similarMovies,watchProviders
0,1,Toy Story (1995),Toy_Story,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,"martial arts, jealousy, friendship, bullying, ...","Police Story, Harlem Nights, Snow White: A Tal...",Disney Plus
1,2,Jumanji (1995),Jumanji,"Robin Williams, Kirsten Dunst, Bradley Pierce,...",Joe Johnston,"giant insect, board game, jungle, disappearanc...","Medicine Man, The Jungle Book, Honey, I Shrunk...","fuboTV, Peacock Premium, AMC Plus Apple TV Cha..."
2,3,Grumpier Old Men (1995),Grumpier_Old_Men,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,"fishing, halloween, sequel, old man, best friend","Young Guns II, Death to Smoochy, The Nutty Pro...",NaN
3,4,Waiting to Exhale (1995),Waiting_to_Exhale,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"based on novel or book, interracial relationsh...","Just Cause, Love Story, The Sword in the Stone...",NaN
4,5,Father of the Bride Part II (1995),Father_of_the_Bride_Part_II,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,"parent child relationship, baby, midlife crisi...","10, Little Man, Young Guns II, S.W.A.T., Welco...","Disney Plus, Hulu"
...,...,...,...,...,...,...,...,...
9571,193581,Black Butler: Book of the Atlantic (2017),Black_Butler:_Book_of_the_Atlantic,"Daisuke Ono, Maaya Sakamoto, Yukari Tamura, Ju...",Noriyuki Abe,"butler, victorian england, based on manga, dar...","Seven Days War, The Time Machine, To Walk Invi...",NaN
9572,193583,No Game No Life: Zero (2017),No_Game_No_Life:_Zero,"Yoko Hikasa, Ai Kayano, Yoshitsugu Matsuoka, K...",Atsuko Ishizuka,"based on novel or book, anti hero, magic, post...","10, Just Cause, Love Story, The Sword in the S...",HiDive
9573,193585,Flint (2017),Flint,"Queen Latifah, Betsy Brandt, Rob Morrow, Marin...",Bruce Beresford,NaN,NaN,NaN
9574,193587,Bungo Stray Dogs: Dead Apple (2018),Bungo_Stray_Dogs:_Dead_Apple,"Kensho Ono, Mamoru Miyano, Kisho Taniyama, Yut...",Takuya Igarashi,"supernatural, mafia, based on manga, super pow...","Donnie Brasco, Thir13en Ghosts, Ghosts of War,...",Funimation Now


In [111]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ast

def transform_list_of_strings(strings):
    transformed_list = [ast.literal_eval(string) for string in strings]
    return transformed_list

def replace_film(movies_list):
    movies = []
    remove = [" film", " (film)"]
    for movie in movies_list:  
        for i in remove:            
            movie = movie.replace(i,"").strip()
        movies.append(movie)    
    return movies

def get_movie_names_from_dbpedia(movie_uris):
    # Create a SPARQLWrapper instance and set the DBpedia endpoint URL
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    movies_names = []
    
    for i in range(0, len(movie_uris), 5):
        movies = movie_uris[i:i+5]
        # Compose the SPARQL query to retrieve the movie names
        query = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?movieName
        WHERE {
          VALUES ?movie {
            %s
          }
          ?movie rdfs:label ?movieName .
          FILTER (lang(?movieName) = 'en')
        }
        """ % ' '.join(['<'+uri+'>' for uri in movies])

        # Set the SPARQL query and request JSON results
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        # Execute the SPARQL query and retrieve the results
        results = sparql.query().convert()

        # Extract the movie names from the results
        movie_names_extract = [result['movieName']['value'] for result in results['results']['bindings']]
        movies_names.extend(movie_names_extract)
    return movies_names  

def get_movie_names_from_dbpedia(movie_uris):
    # Create a SPARQLWrapper instance and set the DBpedia endpoint URL
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
   
    # Compose the SPARQL query to retrieve the movie names
    query = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?movieName
    WHERE {
      VALUES ?movie {
        %s
      }
      ?movie rdfs:label ?movieName .
      FILTER (lang(?movieName) = 'en')
    }
    """ % ' '.join(['<'+uri+'>' for uri in movie_uris])

    # Set the SPARQL query and request JSON results
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the SPARQL query and retrieve the results
    results = sparql.query().convert()

    # Extract the movie names from the results
    movies_names = [result['movieName']['value'] for result in results['results']['bindings']]
    return movies_names  

def get_similar_movies_by_starring(movie_title, actors_names):
    # Create a SPARQLWrapper instance and set the DBpedia endpoint URL
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    
    actors_names = [string.replace(" ", "_") for string in actors_names]
  
    # Compose the SPARQL query to retrieve similar movies based on shared actors
    query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>

    SELECT DISTINCT ?similarMovie
    WHERE {
      ?similarMovie dbo:starring ?actor .
      FILTER (?actor IN (%s))
      FILTER (?similarMovie != dbr:%s)
    }
    ORDER BY RAND()
    LIMIT 100
    """ % (", ".join(["dbr:" + actor_name for actor_name in actors_names]), movie_title)
    
    # Set the SPARQL query and request JSON results
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the SPARQL query and retrieve the results
    results = sparql.query().convert()

    # Extract the similar movie URIs from the results
    similar_movies = [result['similarMovie']['value'] for result in results['results']['bindings']]
    similar_movies = get_movie_names_from_dbpedia(similar_movies)
    return similar_movies

def get_similar_movies_by_director(movie_title, director_name):
    # Create a SPARQLWrapper instance and set the DBpedia endpoint URL
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    
    director_name = director_name.strip().replace(" ", "_")
     
    # Compose the SPARQL query to retrieve similar movies based on shared actors
    query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>

    SELECT DISTINCT ?similarMovie
    WHERE {
      ?similarMovie dbo:director ?director .
      FILTER (?director = dbr:%s)
      FILTER (?similarMovie != dbr:%s)
    }
    ORDER BY RAND()
    LIMIT 50
    """ % (director_name, movie_title)

    # Set the SPARQL query and request JSON results
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the SPARQL query and retrieve the results
    results = sparql.query().convert()

    # Extract the similar movie URIs from the results
    similar_movies = [result['similarMovie']['value'] for result in results['results']['bindings']]
    similar_movies = get_movie_names_from_dbpedia(similar_movies)
    return similar_movies

In [119]:
title = 'Blade Runner 2049 (2017)'
director = 'Denis Villeneuve'
actors = ['Ryan Gosling', 'Harrison Ford', 'Ana de Armas', 'Jon Voight', 'Tom Sizemore']

# Iterate over the DataFrame and call the get_similar_movies function for each movie
def get_ontological_data(movie_title, director_name, actors_names):
    similar_movies_starring = get_similar_movies_by_starring(movie_title, actors_names)
    similar_movies_actors = get_similar_movies_by_director(movie_title, director)    
    similar_movies_starring.extend(similar_movies_actors)
    
    similar_movies = replace_film(list(set(similar_movies_starring)))
    
    # Create a DataFrame with a single column
    df = pd.DataFrame({'similar_movies': similar_movies})
    
    return df

def get_tmbd_id(movie):
    try:
        search = tmdb.Search()
        response = search.movie(query=movie)
        response_id = response["results"][0]['id']
        return response_id
    except:
        return np.nan    
    
def join_ontological_data(df_ontological):    
    df_ontological['tmdbId'] = np.nan
    df_ontological['tmdbId'] = df_ontological['similar_movies'].apply(lambda x: get_tmbd_id(x))
    df_ontological['tmdbId'] = df_ontological['tmdbId'].fillna(0).astype(int)
    
    columns = ["tmdbId", "movieId", "title", "actors", "director", "keyWords", "similarMovies", "watchProviders"]

    df_merged = df_ontological.merge(df_l[['tmdbId', 'movieId']], on='tmdbId', how='left')
    df_merged = df_merged.dropna()
    df_merged = df_merged.merge(df_m_extended, on='movieId', how='left')
    df_merged = df_merged[columns]
    df_merged = df_merged.loc[df_merged['actors'].isna() == False]
    df_merged['movieId'] = df_merged['movieId'].astype(int)
    return df_merged
    
def knn_predict(user_id, item_id):
    knn_prediction = knn.predict(user_id, item_id).est
    return knn_prediction

In [115]:
# Cargamos los modelos
from surprise import dump
knn = dump.load('knn.pkl')[1]
df = df_ratings[['userId', 'movieId', 'rating']]

In [138]:
import random
random.seed(seed)

def get_top_n(user_id, movie_id, n, sample = 1):   
    movie = df_m_extended.loc[df_m_extended['movieId'] == movie_id] # Movie that the users watches
    title = preprocess_movie_name(movie['title'].values[0])
    director = movie['director'].values[0]
    actors = [value.strip() for value in movie['actors'].values[0].split(',')]
       
    # Ontological filtering
    df_ontological = get_ontological_data(title, director, actors)
    df_final = join_ontological_data(df_ontological)
    
    print("Data retrieved:", len(df_final))
    print()
    
    # Get all businesses the user has not reviewed
    all_items = set(df_final['movieId'].unique())
    reviewed_items = set(df.loc[df['userId'] == user_id]['movieId'].unique())
    unreviewed_items = all_items - reviewed_items
    
    print("Ontological filtered movies:", all_items)
    print()
    print("Reviewed:",reviewed_items)
    print()
    print("Unreviewed:", unreviewed_items)

    print()
    print("Total movies unreviewed:", len(unreviewed_items))
    print()
   
    unreviewed_items = list(unreviewed_items)
    num_items = len(unreviewed_items)
    num_sample = int(num_items * sample)
    unreviewed_items = random.sample(unreviewed_items, num_sample)
    
    print("Predicting movies...")
    
    # Make predictions for unreviewed businesses and sort by prediction
    predictions = []
    for item_id in unreviewed_items:
        prediction = knn_predict(user_id, item_id)
        predictions.append((item_id, prediction))
        #print((business_id, prediction))
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    print("Retrieving data from DBPedia...")
    print()
    
    print("User:", user_id)
    print()
    print("Movie title:", movie['title'].values[0])
    print("Director:", director)
    print("Actors:", actors)
    print()

    # Get top n recommended businesses and their predictions
    top_n = predictions[:n]
    top_n_items = [x[0] for x in top_n]
    top_n_predictions = [x[1] for x in top_n]
    top_n_percentage = [str(round(x/5*100, 2))+"%" for x in top_n_predictions]

    # Create DataFrame with business ids and predictions
    columns = ["tmdbId", "movieId", "title", "actors", "director", "keyWords", "similarMovies", "watchProviders"]

    df_top_n = pd.DataFrame({'movieId': top_n_items, 'prediction': top_n_predictions, 'percentage': top_n_percentage})
    df_top_n =  pd.merge(df_top_n, df_final, on='movieId')
    return df_top_n

# Example usage of hybrid predictor
n = 10
user_id = 5
movie_id = 257
top_n_movies = get_top_n(user_id, movie_id, n)
top_n_movies

Data retrieved: 47

Ontological filtered movies: {103042, 45186, 2948, 2947, 517, 2181, 50954, 1275, 3984, 529, 2194, 1303, 92439, 6298, 43677, 44709, 809, 5033, 6057, 2605, 46, 2993, 4019, 1590, 1208, 168248, 3513, 1601, 1610, 4812, 6990, 5071, 164179, 6996, 6870, 53464, 6365, 3423, 3424, 358, 998, 2153, 176371, 139644, 2555, 7292, 86781}

Reviewed: {1, 515, 261, 265, 266, 527, 531, 21, 150, 534, 153, 410, 34, 290, 36, 39, 296, 300, 50, 58, 316, 318, 457, 588, 589, 590, 592, 594, 595, 596, 597, 344, 474, 475, 349, 608, 357, 232, 364, 110, 367, 247, 380, 253}

Unreviewed: {103042, 2947, 2948, 517, 45186, 2181, 50954, 3984, 529, 2194, 1303, 92439, 6298, 43677, 44709, 809, 5033, 6057, 2605, 46, 2993, 4019, 1590, 2555, 1208, 168248, 3513, 1601, 1610, 4812, 6990, 5071, 7292, 164179, 6996, 6870, 53464, 6365, 3423, 3424, 358, 998, 2153, 176371, 1275, 139644, 86781}

Total movies unreviewed: 47

Predicting movies...
Retrieving data from DBPedia...

User: 5

Movie title: Just Cause (1995)
Dire

,movieId,prediction,percentage,tmdbId,title,actors,director,keyWords,similarMovies,watchProviders
0,86781,4.247920,84.96%,46738,Incendies (2010),"Lubna Azabal, Mélissa Désormeaux-Poulin, Maxim...",Denis Villeneuve,"prison, middle east, rape, muslim, militia","Just Cause, Young Guns II, The Longest Yard, T...",NaN
1,168248,4.156063,83.12%,324552,John Wick: Chapter Two (2017),"Keanu Reeves, Common, Ian McShane, Laurence Fi...",Chad Stahelski,"italy, gun, roof, hitman, secret organization","Only You, Young Guns II, Eraser, 11:14, The Av...",NaN
2,1303,4.118552,82.37%,983,"Man Who Would Be King, The (1975)","Sean Connery, Michael Caine, Christopher Plumm...",John Huston,"robbery, journalist, gold, cheating, treasure","Tristan & Isolde, Only You, Harlem Nights, The...",NaN
3,2181,4.114334,82.29%,506,Marnie (1964),"Tippi Hedren, Sean Connery, Martin Gabel, Dian...",Alfred Hitchcock,"philadelphia, pennsylvania, post traumatic str...","Tristan & Isolde, 10, Police Story, Tales from...","Netflix, Netflix basic with Ads"
4,3424,4.077714,81.55%,925,Do the Right Thing (1989),"Danny Aiello, Ossie Davis, Ruby Dee, Richard E...",Spike Lee,"new york city, black people, italian american,...","1:1 Thierry Henry, 10, Just Cause, To Wong Foo...",NaN
5,2194,4.052778,81.06%,117,"Untouchables, The (1987)","Kevin Costner, Sean Connery, Charles Martin Sm...",Brian De Palma,"chicago, illinois, prohibition era, gangster, ...","Police Story, Hudson Hawk, Donnie Brasco, Ferr...","fuboTV, Paramount Plus, Paramount Plus Apple T..."
6,5071,4.000000,80.0%,35650,Maelström (2000),"Marie-Josée Croze, Jean-Nicolas Verreault, Ste...",Denis Villeneuve,abortion,"Women Is Losers, Unpregnant, First Reformed, L...",NaN
7,2947,3.998311,79.97%,658,Goldfinger (1964),"Sean Connery, Honor Blackman, Gert Fröbe, Shir...",Guy Hamilton,"england, based on novel or book, golf, secret ...","Leave the World Behind, Just Cause, Love Story...","Paramount Plus Apple TV Channel , MGM Plus Rok..."
8,1610,3.984626,79.69%,1669,"Hunt for Red October, The (1990)","Sean Connery, Alec Baldwin, Scott Glenn, Sam N...",John McTiernan,"submarine, spy, based on novel or book, cold w...","Confidential Assignment, Just Cause, Love Stor...","AMC Plus Apple TV Channel , AMC+"
9,44709,3.909736,78.19%,13751,Akeelah and the Bee (2006),"Keke Palmer, Laurence Fishburne, Angela Basset...",Doug Atchison,"black people, spelling, spelling bee","Just Cause, American Gangster, Bee Season, How...",Peacock Premium
